# 패키지 다운

In [1]:
#기본 패키지
import time
import os 
import glob
import warnings
warnings.filterwarnings('ignore')

#데이터 처리 패키지
import pandas as pd
import numpy as np

#시각화 패키지
import matplotlib as mpl
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from matplotlib import rc 
rc('font', family='malgun gothic')
rc('axes', unicode_minus = False)
import seaborn as sns
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')

#수학 패키지
from scipy.stats.mstats import gmean
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
import uuid
import random
from tqdm import tqdm
from itertools import combinations

#머신러닝 패키지
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import *
from sklearn.naive_bayes import GaussianNB 
from sklearn.linear_model import LogisticRegression, SGDClassifier, LinearRegression, SGDRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from vecstack import stacking, StackingTransformer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [2]:
from pathlib import Path

# 데이터 불러오기

In [3]:
data_path_1 = os.path.abspath('../data/01_제공데이터')
data_path_2 = os.path.abspath('../data/02_평가데이터')
data_path_3 = os.path.abspath('../data/03_수집데이터')

In [4]:
# 3-8월까지 크롤링해온 데이터에 고유선수코드까지 전처리 완료한 데이터 
crawl = pd.read_csv(data_path_3+"/statiz_rowdata_pcode.csv")

In [5]:
csv_files = glob.glob(data_path_1+'/*.csv')

dataset = {}    # Raw data Dick 자징하기
files = []      # Raw data file 리스트 
for name in csv_files:
    raw = pd.read_csv(name,encoding = "cp949")
    name = os.path.basename(name).split('.')[0] # 파일의 확장자 제외 이름만 가져오기 
    dataset.update({name : raw})    # 데이터 사전 

    files.append(name)  

In [6]:
hit_2018 = dataset[files[0]]
hit_2019 = dataset[files[1]]
hit_2020 = dataset[files[2]]
hit_2021 = dataset[files[3]]

player_2018 = dataset[files[5]]
player_2019 = dataset[files[6]]
player_2020 = dataset[files[7]]
player_2021 = dataset[files[8]]

batter_2018 = dataset[files[9]]
batter_2019 = dataset[files[10]]
batter_2020 = dataset[files[11]]
batter_2021 = dataset[files[12]]

In [7]:
# stat_nn 은 각 년도별 9-10월의 출루율(정답데이터)
stat_18 = pd.read_csv(data_path_3+'/statiz2_18.csv',index_col = 0)
stat_19 = pd.read_csv(data_path_3+'/statiz2_19.csv',index_col = 0)
stat_20 = pd.read_csv(data_path_3+'/statiz2_18.csv',index_col = 0)

In [8]:
# 크롤링 데이터를 465경기까지만 수집한 데이터
statiz_row = pd.read_csv(data_path_3+"/statiz_465_row_data.csv")

# 데이터 전처리

In [9]:
crawl['타석'] = crawl['타수']+ crawl['볼넷'] + crawl['사구'] + crawl['희타']+ crawl['희비']

In [10]:
# 크롤링해온 데이터에서 109타석이상인 선수를 고유선수코드를 기준으로 정리
crawl_gb = crawl.groupby('고유선수코드')['타석','타수','안타','2타','3타','홈런','루타','볼넷','사구','고의사구','삼진','선발','도루','희타','희비','도실'].sum().reset_index()

In [11]:
# 영어 column을 가독성 좋게 한글로 변경
hit_2018.columns = ["년도", "경기코드", "투구코드", "선수코드", "팀코드", "이닝", "타구속도","발사각도", "타구종류", "구속", "경기장"]
hit_2019.columns = ["년도", "경기코드", "투구코드", "선수코드", "팀코드", "이닝", "타구속도","발사각도", "타구종류", "구속", "경기장"]
hit_2020.columns = ["년도", "경기코드", "투구코드", "선수코드", "팀코드", "이닝", "타구속도","발사각도", "타구종류", "구속", "경기장"]
hit_2021.columns = ["년도", "경기코드", "투구코드", "선수코드", "팀코드", "이닝", "타구속도","발사각도", "타구종류", "구속", "경기장"]

player_2018.columns = ["년도", "선수코드", "이름", "팀코드", "포지션", "나이", "연봉"]
player_2019.columns = ["년도", "선수코드", "이름", "팀코드", "포지션", "나이", "연봉"]
player_2020.columns = ["년도", "선수코드", "이름", "팀코드", "포지션", "나이", "연봉"]
player_2021.columns = ["년도", "선수코드", "이름", "팀코드", "포지션", "나이", "연봉"]

batter_2018.columns = ["년도", "선수코드", "출장경기수", "타석", "타수", "타율", "안타", "홈런", "루타", "장타율", "희생플라이", "볼넷", "삼진", "고의사구", "사구", "병살타"]
batter_2019.columns = ["년도", "선수코드", "출장경기수", "타석", "타수", "타율", "안타", "홈런", "루타", "장타율", "희생플라이", "볼넷", "삼진", "고의사구", "사구", "병살타"]
batter_2020.columns = ["년도", "선수코드", "출장경기수", "타석", "타수", "타율", "안타", "홈런", "루타", "장타율", "희생플라이", "볼넷", "삼진", "고의사구", "사구", "병살타"]
batter_2021.columns =["년도", "선수코드", "출장경기수", "타석", "타수", "타율", "안타", "홈런", "루타", "장타율", "희생플라이", "볼넷", "삼진", "고의사구", "사구", "병살타"]

In [12]:
# 출루, 출루율, OPS를 새로 만들어줌
batter_2018["출루"] = batter_2018["안타"] + batter_2018["볼넷"] + batter_2018["사구"] 
batter_2019["출루"] = batter_2019["안타"] + batter_2019["볼넷"] + batter_2019["사구"] 
batter_2020["출루"] = batter_2020["안타"] + batter_2020["볼넷"] + batter_2020["사구"] 
batter_2021["출루"] = batter_2021["안타"] + batter_2021["볼넷"] + batter_2021["사구"] 

batter_2018["출루율"] = batter_2018["출루"] / (batter_2018["타수"]+batter_2018["볼넷"]+batter_2018["사구"] + batter_2018["희생플라이"] )
batter_2019["출루율"] = batter_2019["출루"] / (batter_2019["타수"]+batter_2019["볼넷"]+batter_2019["사구"] + batter_2019["희생플라이"] )
batter_2020["출루율"] = batter_2020["출루"] / (batter_2020["타수"]+batter_2020["볼넷"]+batter_2020["사구"] + batter_2020["희생플라이"] )
batter_2021["출루율"] = batter_2021["출루"] / (batter_2021["타수"]+batter_2021["볼넷"]+batter_2021["사구"] + batter_2021["희생플라이"] )

batter_2018["OPS"] = batter_2018["장타율"]+ batter_2018["출루율"]
batter_2019["OPS"] = batter_2019["장타율"]+ batter_2019["출루율"]
batter_2020["OPS"] = batter_2020["장타율"]+ batter_2020["출루율"]
batter_2021["OPS"] = batter_2021["장타율"]+ batter_2021["출루율"]

In [13]:
# 384 경기 기준으로 스플릿
game_2018 = hit_2018.경기코드.unique()[:384]
game_2019 = hit_2019.경기코드.unique()[:383]
game_2020 = hit_2020.경기코드.unique()[:384]

In [14]:
train_game = pd.concat([hit_2020.query("경기코드 in @game_2020"), hit_2019.query("경기코드 in @game_2019"), hit_2018.query("경기코드 in @game_2018")]).reset_index(drop = True)
test_game = hit_2021

In [15]:
# 배럴피쳐를 만들기 위한 데이터셋
tr = pd.concat([train_game, test_game]).reset_index(drop = True)

In [16]:
# 고유선수코드(년도+선수코드)를 나눠진 데이터들의 키값으로 사용
tr["고유선수코드"] = tr["년도"].astype("str")+ "_" + tr["선수코드"].astype("str")

# 피쳐 생성1

In [17]:
inplay = ["내야안타(1루타)", "1루타", "2루타", "3루타", "홈런"]
planplay = ["번트안타", "희생번트", "희생플라이", "번트아웃"]
outplay = ["플라이", "땅볼아웃", "파울플라이", "병살타", "삼중살타", "직선타", "인필드플라이", "야수선택"]

In [18]:
tr["인플레이타구"] = tr.apply(lambda x: 1
                                    if (x["타구종류"] in inplay)
                                    else 0, axis = 1)

tr["작전수행타구"] = tr.apply(lambda x: 1
                                    if (x["타구종류"] in planplay)
                                    else 0, axis = 1)

tr["아웃플레이타구"] = tr.apply(lambda x: 1
                                    if (x["타구종류"] in outplay)
                                    else 0, axis = 1)
tr['배럴각도타구'] = tr.apply(lambda x: 1 if (22 < x['발사각도'] < 30) else 0, axis = 1)
tr['배럴속도타구'] = tr.apply(lambda x: 1 if (140 < x['타구속도']) else 0, axis = 1)

In [19]:
features =pd.DataFrame({"고유선수코드" : crawl_gb.고유선수코드.unique()})

In [20]:
crawl_gb['타수/타석'] = crawl_gb['타수'] / crawl_gb['타석']
features = pd.merge(left = features, right= crawl_gb[['고유선수코드','타수/타석']], on='고유선수코드')

In [21]:
f = tr.groupby(['고유선수코드'])['배럴각도타구'].agg([('배럴각도타구수', 'sum' )]).reset_index()
features = pd.merge(left = features, right = f, on='고유선수코드' )

In [22]:
f = tr.groupby(['고유선수코드'])['배럴속도타구'].agg([('배럴속도타구수', 'sum' )]).reset_index()
features = pd.merge(left = features, right = f, on='고유선수코드' )

In [23]:
f = tr.groupby(["고유선수코드"])["경기코드"].agg([("출장경기수",lambda x : len(x.unique()))]).reset_index()
features = pd.merge(left = features, right = f,on = "고유선수코드")

In [24]:
f = tr.groupby(["고유선수코드"])["타구속도"].agg([("평균타구속도", "mean" )]).reset_index()
features = pd.merge(left = features, right = f,on = "고유선수코드")

In [25]:
f = tr.groupby(["고유선수코드"])["인플레이타구"].agg([("인플레이타구수","sum")]).reset_index()
features = pd.merge(left = features, right = f,on = "고유선수코드")

In [26]:
f = tr.groupby(["고유선수코드"])["아웃플레이타구"].agg([("아웃플레이타구수","sum")]).reset_index()
features = pd.merge(left = features, right = f,on = "고유선수코드")

In [27]:
f = tr.groupby(["고유선수코드"])["작전수행타구"].agg([("작전수행타구수","sum")]).reset_index()
features = pd.merge(left = features, right = f,on = "고유선수코드")

In [28]:
features['작전수행타구수'] = features['작전수행타구수'].astype('float64')

In [29]:
player = pd.concat([player_2018,player_2019, player_2020, player_2021]).reset_index(drop = True)
player["고유선수코드"] = player["년도"] .astype("str") + "_" + player["선수코드"].astype("str")

In [30]:
f = player.loc[:,["고유선수코드","나이"]]
features = pd.merge(left = features, right = f,on = "고유선수코드")

In [31]:
f = player.loc[:,["고유선수코드","팀코드"]]
features = pd.merge(left = features, right = f,on = "고유선수코드")

In [32]:
tr2=crawl_gb.groupby('고유선수코드')['타수'].sum().reset_index()
features = pd.merge(left = features ,right = tr2 , on = "고유선수코드")

In [33]:
tr2=crawl_gb.groupby('고유선수코드')['도루'].sum().reset_index()
features = pd.merge(left = features ,right = tr2 , on = "고유선수코드")

In [34]:
tr2=crawl_gb.groupby('고유선수코드')['도실'].sum().reset_index()
features = pd.merge(left = features ,right = tr2 , on = "고유선수코드")

In [35]:
tr2=crawl_gb.groupby('고유선수코드')['고의사구'].sum().reset_index()
features = pd.merge(left = features ,right = tr2 , on = "고유선수코드")

In [36]:
tr2=crawl_gb.groupby('고유선수코드')['볼넷'].sum().reset_index()
features = pd.merge(left = features ,right = tr2 , on = "고유선수코드")

In [37]:
tr2=crawl_gb.groupby('고유선수코드')['선발'].sum().reset_index()
features = pd.merge(left = features ,right = tr2 , on = "고유선수코드")

In [38]:
tr2=crawl.groupby('고유선수코드')['일자'].count().reset_index()
features = pd.merge(left = features ,right = tr2 , on = "고유선수코드")

In [39]:
tr2=crawl_gb.groupby('고유선수코드')['삼진'].sum().reset_index()
features = pd.merge(left = features ,right = tr2 , on = "고유선수코드")

In [40]:
tr2=crawl_gb.groupby('고유선수코드')['희타'].sum().reset_index()
features = pd.merge(left = features ,right = tr2 , on = "고유선수코드")

In [41]:
tr2=crawl_gb.groupby('고유선수코드')['희비'].sum().reset_index()
features = pd.merge(left = features ,right = tr2 , on = "고유선수코드")

In [42]:
tr2 = tr.copy()

## 배럴 피쳐

In [43]:
tr2["안타라벨"] = tr2.apply(lambda x: 1
                                    if (x["타구종류"] in inplay)
                                    else 0, axis = 1)

In [44]:
def hit_vel(x):
    y = int(x)
    z = y%2
    a = y-z
    return a

In [45]:
def hit_angle(x):
    y = int(x)
    z = y%2
    a = y-z
    return a

In [46]:
def hit_result(x):
    if x == '1루타' :
        return 1
    elif x == '2루타' :
        return 2
    elif x == '3루타' :
        return 3
    elif x == '홈런' :
        return 4
    else:
        return  0
    

In [47]:
tr2['타구속도 범위']=tr2.타구속도.apply(lambda x: hit_vel(x))
tr2['타구각도 범위']=tr2.발사각도.apply(lambda x: hit_angle(x))
tr2['타구결과']=tr2.타구종류.apply(lambda x: hit_result(x))

In [48]:
tr2['key'] = tr2['타구속도 범위'].astype(str)+"/"+tr2['타구각도 범위'].astype(str)

In [49]:
key_value = tr2.groupby("key")["안타라벨"].agg([("안타수","sum")])
key_value["타구의수"] = tr2.groupby("key")["key"].agg([("타구의수","size")])
key_value["타구결과의 합"] = tr2.groupby("key")["타구결과"].agg([("장타율합","sum")])

In [50]:
key_value["타율"] = key_value["안타수"]/key_value["타구의수"]
key_value["장타율"] = key_value["타구결과의 합"]/key_value["타구의수"]

In [51]:
key_value.query("장타율 >= 3.7")

,안타수,타구의수,타구결과의 합,타율,장타율
key,,,,,
158/50,1,1,4,1.000000,4.000000
160/24,45,46,174,0.978261,3.782609
160/26,31,32,123,0.968750,3.843750
160/30,18,19,72,0.947368,3.789474
162/22,27,27,104,1.000000,3.851852
162/24,27,27,106,1.000000,3.925926
162/26,32,32,128,1.000000,4.000000
162/28,32,33,126,0.969697,3.818182
162/30,14,15,56,0.933333,3.733333


In [52]:
key_value_2 = key_value.reset_index()

In [53]:
key_value_2["타구속도"] = key_value_2["key"].apply(lambda x : x.split("/")[0])
key_value_2["발사각도"] = key_value_2["key"].apply(lambda x : x.split("/")[1])

In [54]:
alpha = key_value_2.loc[(key_value_2.타율 >= 0.5)&(key_value_2.장타율>=1.5)]

In [55]:
alpha1 = alpha.drop(alpha.index[alpha.발사각도.str.contains('-')])

In [56]:
alpha2 = alpha1[['안타수','타구의수','타구결과의 합','타율','장타율','타구속도','발사각도']].astype('float64') 

In [57]:
alpha3 = alpha2.loc[(alpha2.타구속도>= 148) & (alpha2.발사각도>0)]

In [58]:
def barrels(a,b):

    m = 148
    n = 26
    if (a - m <= 2) &  (2 >= n-b >-1):
        return 100
    elif (2< a - m <= 4) &  (4>= n-b >-2):
        return 100
    elif (2< a - m <= 6) &  (8 >=n-b>-4):
        return 120
    elif (2< a - m <= 12) &  (12 >=n-b> -6):
        return 160
    elif (2< a - m <= 16) &  (12>= n-b >-8):
        return 180
    elif (2< a - m <= 20) &  (14>= n-b >-8):
        return 180
    elif (2< a - m <= 40) &  (18>= n-b >-10):
        return 200
    else:
        return 0


In [59]:
tr2['가중배럴']=tr2.apply(lambda x: barrels(x['타구속도'], x['발사각도']),axis=1)

In [60]:
tr3 = tr2.groupby('고유선수코드')['가중배럴'].sum().reset_index()
features = pd.merge(left = features ,right = tr3 , on = "고유선수코드")

In [61]:
def bb(a):
    if a == 0:
        return 0
    else:
        return 1

In [62]:
tr2['가중배럴01'] = tr2.apply(lambda x: bb(x['가중배럴']),axis=1)
tr2 = tr2.rename(columns = {'가중배럴01':'가중배럴카운트'})

In [63]:
tr2 = tr2.groupby('고유선수코드')['가중배럴카운트'].sum().reset_index()

In [64]:
features = pd.merge(left = features ,right = tr2 , on = "고유선수코드")

# 피쳐 생성2

In [65]:
batter_ = pd.concat([batter_2018,batter_2019,batter_2020,batter_2021]).reset_index(drop = True)

batter_["고유선수코드"] = batter_["년도"].astype("str") + "_" +batter_["선수코드"].astype("str")

In [66]:
player = pd.concat([player_2018,player_2019, player_2020, player_2021]).reset_index(drop = True)
player["고유선수코드"] = player["년도"] .astype("str") + "_" + player["선수코드"].astype("str")

In [67]:
statiz_row.loc[statiz_row["선수"] == "한유섬",["선수"]] = "한동민"
statiz_row.loc[(statiz_row["선수"] == "한동민") & (statiz_row["년도"] == 2021),["선수"]] = "한유섬"

In [68]:
statiz_row["년도이름"] = statiz_row["년도"].astype("str") + "_" + statiz_row["선수"]

In [69]:
df = batter_[batter_.타석 >= 70].reset_index(drop = True)

In [70]:
df = pd.merge(left = df, right = player.loc[:,["고유선수코드","이름"]],on = "고유선수코드")

In [71]:
df["년도이름"] = df["년도"].astype('str') + "_" + df["이름"]

In [72]:
statiz_row = pd.merge(left = statiz_row, right = df.loc[:,["고유선수코드","년도이름"]], on = "년도이름")

In [73]:
year = 2018
lst = [stat_18, stat_19, stat_20]
for i in lst:
    i["년도"] = year
    i["년도이름"] = str(year) + "_" + i["선수"]
    year = year + 1

In [74]:
sta_y = pd.concat([stat_18, stat_19, stat_20], axis = 0).reset_index(drop = True)

In [75]:
sta_y.loc[sta_y["선수"] == "한유섬",["선수"]] = "한동민"
sta_y.loc[(sta_y["선수"] == "한동민") & (sta_y["년도"] == 2021),["선수"]] = "한유섬"

In [76]:
sta_y["타석"] = sta_y["타수"] + sta_y["볼넷"] + sta_y["사구"] + sta_y["희비"] + sta_y["희타"]

In [77]:
sta_y["년도이름"] = sta_y["년도"].astype("str") + "_" + sta_y["선수"]

In [78]:
sta_y = sta_y[sta_y.타석 >= 31].reset_index(drop = True)

In [79]:
sta_y["ops"] = sta_y["출루율"] + sta_y["장타율"]

In [80]:
lst = [2018,2019,2020]
sta_train = statiz_row.query("년도 in @lst")

In [81]:
lst = sta_train["년도이름"].unique()
sta_y = sta_y.query("년도이름 in @lst").reset_index(drop = True)

In [82]:
t = sta_train.groupby("년도이름")["고유선수코드"].agg([("고유선수코드","first")]).reset_index()

In [83]:
sta_y = pd.merge(left = sta_y , right = t, on ="년도이름")

In [84]:
sta_y = sta_y.loc[:,["고유선수코드","선수","출루율","장타율","ops"]] # 타겟 데이터

In [85]:
features1 =pd.DataFrame({"고유선수코드" : statiz_row.고유선수코드.unique()})

In [86]:
statiz_row["월"] = statiz_row["일자"].apply(lambda x:x[:2]).astype("str")

In [87]:
statiz_row["일"] = statiz_row["일자"].apply(lambda x:x[3:]).astype("str")

In [88]:
def str_cut(seq, length):
    return [''.join(x) for x in zip(*[list(seq[z::length]) for z in range(length)])]

In [89]:
statiz_row_2018 = statiz_row[statiz_row.년도 == 2018]

f = statiz_row_2018.groupby(["월"])["일"].agg([('일자문자',"sum")]).reset_index()

f.일자문자 = f.일자문자.apply(lambda x: str_cut(x,2))

sort_list = []
for i in range(len(f.일자문자)):
    sort_list.append(sorted(list(set(f.일자문자[i]))))

sort_list_2 = []
for i in range(len(f.일자문자)):
    for j in range(len(sort_list[i])):
        day = f.월[i] + "-" + sort_list[i][j]
        sort_list_2.append(day)

sort_dic = dict()
for i in range(len(sort_list_2)):
    sort_dic[sort_list_2[i]] = str(i)

statiz_row_2018["교정일자"] = statiz_row_2018.일자.apply(lambda x:sort_dic[x])

In [90]:
statiz_row_2019 = statiz_row[statiz_row.년도 == 2019]

f = statiz_row_2019.groupby(["월"])["일"].agg([('일자문자',"sum")]).reset_index()

f.일자문자 = f.일자문자.apply(lambda x: str_cut(x,2))

sort_list = []
for i in range(len(f.일자문자)):
    sort_list.append(sorted(list(set(f.일자문자[i]))))

sort_list_2 = []
for i in range(len(f.일자문자)):
    for j in range(len(sort_list[i])):
        day = f.월[i] + "-" + sort_list[i][j]
        sort_list_2.append(day)

sort_dic = dict()
for i in range(len(sort_list_2)):
    sort_dic[sort_list_2[i]] = str(i)

statiz_row_2019["교정일자"] = statiz_row_2019.일자.apply(lambda x:sort_dic[x])

In [91]:
statiz_row_2020 = statiz_row[statiz_row.년도 == 2020]

f = statiz_row_2020.groupby(["월"])["일"].agg([('일자문자',"sum")]).reset_index()

f.일자문자 = f.일자문자.apply(lambda x: str_cut(x,2))

sort_list = []
for i in range(len(f.일자문자)):
    sort_list.append(sorted(list(set(f.일자문자[i]))))

sort_list_2 = []
for i in range(len(f.일자문자)):
    for j in range(len(sort_list[i])):
        day = f.월[i] + "-" + sort_list[i][j]
        sort_list_2.append(day)

sort_dic = dict()
for i in range(len(sort_list_2)):
    sort_dic[sort_list_2[i]] = str(i)

statiz_row_2020["교정일자"] = statiz_row_2020.일자.apply(lambda x:sort_dic[x])

In [92]:
statiz_row_2021 = statiz_row[statiz_row.년도 == 2021]

f = statiz_row_2021.groupby(["월"])["일"].agg([('일자문자',"sum")]).reset_index()

f.일자문자 = f.일자문자.apply(lambda x: str_cut(x,2))

sort_list = []
for i in range(len(f.일자문자)):
    sort_list.append(sorted(list(set(f.일자문자[i]))))

sort_list_2 = []
for i in range(len(f.일자문자)):
    for j in range(len(sort_list[i])):
        day = f.월[i] + "-" + sort_list[i][j]
        sort_list_2.append(day)

sort_dic = dict()
for i in range(len(sort_list_2)):
    sort_dic[sort_list_2[i]] = str(i)

statiz_row_2021["교정일자"] = statiz_row_2021.일자.apply(lambda x:sort_dic[x])

In [93]:
statiz_row = pd.concat([statiz_row_2018, statiz_row_2019, statiz_row_2020, statiz_row_2021])

In [94]:
statiz_row["교정일자"] = statiz_row["교정일자"].astype("int")

In [95]:
def day_split(x):
        if x <10:
            return 1
        elif x<20:
            return 2
        elif x<30:
            return 3
        elif x<40:
            return 4
        elif x<50:
            return 5
        elif x<60:
            return 6
        elif x<70:
            return 7
        elif x<80:
            return 8
        elif x<90:
            return 9
        else:
            return 10

In [96]:
statiz_row["교정주기"] = statiz_row["교정일자"].apply(lambda x: day_split(x))

In [97]:
p = pd.DataFrame({"고유선수코드" : statiz_row.고유선수코드.unique()})
for i in list(set(statiz_row["교정주기"].unique())):
    statiz_ten = statiz_row[statiz_row.교정주기 == i].reset_index(drop = True)
    f = pd.DataFrame({"고유선수코드" : statiz_row.고유선수코드.unique()})

    t = statiz_ten.groupby("고유선수코드")["타석"].agg([("타석","sum")]).reset_index()
    f = pd.merge(left = f, right = t ,how = "outer", on = "고유선수코드")
    
    t = statiz_ten.groupby("고유선수코드")["안타"].agg([("안타","sum")]).reset_index()
    f = pd.merge(left = f, right = t ,how = "outer" , on = "고유선수코드")

    t = statiz_ten.groupby("고유선수코드")["볼넷"].agg([("볼넷","sum")]).reset_index()
    f = pd.merge(left = f, right = t ,how = "outer", on = "고유선수코드")

    t = statiz_ten.groupby("고유선수코드")["사구"].agg([("사구","sum")]).reset_index()
    f = pd.merge(left = f, right = t ,how = "outer", on = "고유선수코드")

    t = statiz_ten.groupby("고유선수코드")["타수"].agg([("타수","sum")]).reset_index()
    f = pd.merge(left = f, right = t ,how = "outer", on = "고유선수코드")

    t = statiz_ten.groupby("고유선수코드")["2타"].agg([("2타","sum")]).reset_index()
    f = pd.merge(left = f, right = t ,how = "outer", on = "고유선수코드")

    t = statiz_ten.groupby("고유선수코드")["3타"].agg([("3타","sum")]).reset_index()
    f = pd.merge(left = f, right = t ,how = "outer", on = "고유선수코드")

    t = statiz_ten.groupby("고유선수코드")["홈런"].agg([("홈런","sum")]).reset_index()
    f = pd.merge(left = f, right = t ,how = "outer", on = "고유선수코드")


    f["{}주기_타율".format(i)] = f["안타"]/f["타수"]
    f["{}주기_출루율".format(i)] = (f["안타"] + f["볼넷"] + f["사구"])/f["타석"]
    f["{}주기_장타율".format(i)] = (f["안타"] + f["2타"] + f["3타"]*2 + f["홈런"]*3)/f["타수"]

    p = pd.merge(left = p, right = f.loc[:,["고유선수코드","{}주기_타율".format(i), "{}주기_출루율".format(i), "{}주기_장타율".format(i)]] , on = "고유선수코드").fillna(0)

In [98]:
features = pd.merge(left = features, right = p,on = "고유선수코드")

In [99]:
statiz_row.head()

,년도,선수,일자,상대,승/패,결과,타순,포지션,선발,타수,득점,안타,2타,3타,홈런,루타,타점,도루,도실,볼넷,사구,고의사구,삼진,병살,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA,월,일,타석,년도이름,고유선수코드,교정일자,교정주기
0,2018,이대호,03-24,@SK,L,5:6,4,DH,1,4,0,2,0,0,0,2,0,0,0,0,0,0,2,0,0,0,0.500,0.500,0.500,1.000,22,1.79,0.79,0.038,03,24,4,2018_이대호,2018_71564,0,1
1,2018,이대호,03-25,@SK,L,0:5,4,1B,1,4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.250,0.250,0.250,0.500,12,0.74,-0.95,-0.076,03,25,4,2018_이대호,2018_71564,1,1
2,2018,이대호,03-27,@두산,L,0:5,4,DH,1,4,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.250,0.250,0.250,0.500,15,0.44,-0.34,-0.004,03,27,4,2018_이대호,2018_71564,2,1
3,2018,이대호,03-28,@두산,L,5:6,4,DH,1,5,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0.235,0.235,0.235,0.471,15,1.16,-0.50,-0.106,03,28,5,2018_이대호,2018_71564,3,1
4,2018,이대호,03-29,@두산,L,1:4,4,DH,1,4,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.238,0.238,0.238,0.476,21,0.99,-0.01,-0.016,03,29,4,2018_이대호,2018_71564,4,1


In [100]:
f = statiz_row.groupby("고유선수코드")["출루"].agg([("최종출루","last")])
features = pd.merge(left = features, right = f,on = "고유선수코드")

In [101]:
f = statiz_row.groupby("고유선수코드")["장타"].agg([("최종장타","last")])
features = pd.merge(left = features, right = f,on = "고유선수코드")

In [102]:
f = statiz_row.groupby("고유선수코드")["타율"].agg([("최종타율","last")])
features = pd.merge(left = features, right = f,on = "고유선수코드")

# 날씨 데이터

In [103]:
weather = pd.read_csv(data_path_3+"/weather.csv")

In [104]:
weather['날짜'] = weather.날짜.apply(lambda x: x.replace('-', ''))

In [105]:
tr["날짜"] = tr.경기코드.apply(lambda x: x[0:8])

In [106]:
tr = pd.merge(tr, weather, on='날짜')
tr['최고기온대비 최저기온'] = tr['최저기온(℃)']/tr['최고기온(℃)']
tr['최고기온대비 평균기온'] = tr['평균기온(℃)']/tr['최고기온(℃)']
tr['최저기온대비 최고기온'] = tr['최고기온(℃)']/tr['최저기온(℃)']
tr['최저기온대비 평균기온'] = tr['평균기온(℃)']/tr['최저기온(℃)']
tr['평균기온대비 최고기온'] = tr['최고기온(℃)']/tr['평균기온(℃)']
tr['평균기온대비 최저기온'] = tr['최저기온(℃)']/tr['평균기온(℃)']

In [107]:
agg_dict = {
    '평균기온(℃)':['min', 'max', 'mean', 'sum', 'median', 'std', 'skew'],
    '최저기온(℃)':['min', 'max', 'mean', 'sum', 'median', 'std', 'skew'],
    '최고기온(℃)':['min', 'max', 'mean', 'sum', 'median', 'std', 'skew'],
    '강수량(mm)':['min', 'max', 'mean', 'sum', 'median', 'std', 'skew'],
    '최고기온대비 최저기온': ['min', 'max', 'mean', 'sum', 'median', 'std', 'skew'],
    '최고기온대비 평균기온': ['min', 'max', 'mean', 'sum', 'median', 'std', 'skew'],
    '최저기온대비 최고기온': ['min', 'max', 'mean', 'sum', 'median', 'std', 'skew'],
    '최저기온대비 평균기온': ['min', 'max', 'mean', 'sum', 'median', 'std', 'skew'],
    '평균기온대비 최고기온': ['min', 'max', 'mean', 'sum', 'median', 'std', 'skew'],
    '평균기온대비 최저기온': ['min', 'max', 'mean', 'sum', 'median', 'std', 'skew']
}

In [108]:
f = tr.groupby('고유선수코드').agg(agg_dict).fillna(0)
f.columns = [('_').join(column) for column in f.columns.ravel()]

In [109]:
f = f.reset_index()
features = pd.merge(left = features, right = f,on = "고유선수코드")
features = features.replace([np.inf, -np.inf], 0)

# 세이버매트릭스

### 타석당 평균 투구수

In [110]:
df = statiz_row.groupby("고유선수코드")["투구"].agg([("투구수","sum")]).reset_index()
df = pd.merge(left = df, right = statiz_row.groupby("고유선수코드")["타석"].agg([("타석","sum")]).reset_index(), on ="고유선수코드")
df["타석당평균투구수"] = df["투구수"] / df["타석"]
features = pd.merge(left = features, right = df.drop(["투구수", "타석"], axis = 1),on = "고유선수코드")

### 삼진비율

In [111]:
df = statiz_row.groupby("고유선수코드")["삼진"].agg([("삼진","sum")]).reset_index()
df = pd.merge(left = df, right = statiz_row.groupby("고유선수코드")["타석"].agg([("타석","sum")]).reset_index(), on ="고유선수코드")
df["삼진비율"] = df["삼진"] / df["타석"]
features = pd.merge(left = features, right = df.drop(["삼진", "타석"], axis = 1),on = "고유선수코드")

### 볼넷비율

In [112]:
df = statiz_row.groupby("고유선수코드")["볼넷"].agg([("볼넷","sum")]).reset_index()
df = pd.merge(left = df, right = statiz_row.groupby("고유선수코드")["타석"].agg([("타석","sum")]).reset_index(), on ="고유선수코드")
df["볼넷비율"] = df["볼넷"] / df["타석"]
features = pd.merge(left = features, right = df.drop(["볼넷", "타석"], axis = 1),on = "고유선수코드")

### ISO (순장타율)

In [113]:
features["iso"] = features["최종장타"] - features["최종타율"]

In [114]:
features["순출루율"] = features["최종출루"] - features["최종타율"]

In [115]:
tr['타구속도 범위']=tr.타구속도.apply(lambda x: hit_vel(x))
tr['타구각도 범위']=tr.발사각도.apply(lambda x: hit_angle(x))

In [116]:
tr['key'] = tr['타구속도 범위'].astype(str)+"/"+tr['타구각도 범위'].astype(str)

In [117]:
batter_list=['148/24','148/26','150/24','150/26','150/28','150/30','150/32','152/22','152/24','152/26',
'152/28','152/30','152/32','152/34','154/20','154/22','154/24','154/26','154/28','154/30',
'154/32','154/34','156/20','156/22','156/24','156/26','156/28','156/30','156/32','156/34',
'158/18','158/20','158/22','158/24','158/26','158/28','158/30','158/32','158/34','158/36',
'160/18','160/20','160/22','160/24','160/26','160/28','160/30','160/32','160/34','160/36',
'160/38','162/18','162/20','162/22','162/24','162/26','162/28','162/30','162/32','162/34',
'162/36','162/38','162/40','164/18','164/20','164/22','164/24','164/26','164/28','164/30',
'164/32','164/34','164/36','164/38','164/40']

In [118]:
batter_list2=[]
for i in range(166,181,2):
    for j in range(16,41,2):
        batter2=str(i)+'/'+str(j)
        batter_list2.append(batter2)

In [119]:
def batter(x):
    if x in batter_list:
        return 1
    elif x in batter_list2:
        return 1

In [120]:
tr['배럴타구여부']=tr.key.apply(lambda x : batter(x))
tr.배럴타구여부.fillna(0,inplace=True)
tr['배럴타구여부']=tr.배럴타구여부.astype(int)

In [121]:
tr2=tr.groupby('고유선수코드')['배럴타구여부'].sum().reset_index()

In [122]:
features = pd.merge(left = features ,right = tr2 , on = "고유선수코드")

In [123]:
features.rename(columns = {'배럴타구여부':'배럴타구갯수'}, inplace=True)

In [124]:
f = tr.groupby(["고유선수코드"])["배럴타구여부"].agg([("배럴타구합","sum")]).reset_index()
f = pd.merge(crawl_gb, f, on='고유선수코드')
f['배럴타구비율'] = f['배럴타구합'] / f['타수']
features = pd.merge(left = features, right = f[['고유선수코드', '배럴타구비율']],on = "고유선수코드")

In [125]:
a = tr.pivot_table(values = "팀코드", index = '고유선수코드', columns = "경기장", aggfunc = "count",fill_value=0).reset_index()
features = pd.merge(left = features ,right = a , on = "고유선수코드")

In [126]:
features['마산_창원']=features['마산']+features['창원']

In [127]:
features.drop(['마산','창원'],axis=1,inplace=True)

In [128]:
tr2 =tr.pivot_table(values = "팀코드", index = '고유선수코드', columns = "이닝", aggfunc = "count",fill_value=0)
features = pd.merge(left = features ,right = tr2 , on = "고유선수코드")

In [129]:
# 257경기를 기준으로 trian, test 분류
End_tr = features[:257]
End_te = features[257:]

In [130]:
# 정답데이터
y_answer = pd.read_csv(data_path_3+'/statiz_all_target.csv')
y_answer

,고유선수코드,선수,출루율,장타율,ops
0,2018_61700,강경학,0.351852,0.175000,0.526852
1,2018_74540,강민호,0.324675,0.375000,0.699675
2,2018_68050,강백호,0.388889,0.537500,0.926389
3,2018_63123,강승호,0.437500,0.622222,1.059722
4,2018_61353,고종욱,0.339623,0.387755,0.727378
...,...,...,...,...,...
252,2020_68525,한동희,0.454545,0.609375,1.063920
253,2020_79240,허경민,0.373333,0.428571,0.801905
254,2020_65399,허정협,0.371429,0.413793,0.785222
255,2020_66108,홍창기,0.460526,0.491803,0.952330


In [131]:
End_tr = pd.merge(left = End_tr , right = y_answer.loc[:,["고유선수코드","출루율"]], on = "고유선수코드")

# 모델링

In [132]:
# Data Wrangling
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import seaborn as sns
%matplotlib inline

# EDA
import klib

# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.base import ClassifierMixin
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.linear_model import LinearRegression

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedKFold
skfold = StratifiedKFold(n_splits=10)

# Utility
import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
import pickle
import joblib
from catboost import CatBoostClassifier

#딥러닝 패키지
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import * #Model
from tensorflow.keras.optimizers import *
from tensorflow.keras.regularizers import *
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.losses import *
from tensorflow.keras import layers

In [133]:
corr = End_tr.drop(['고유선수코드'],1,).corr()["출루율"].abs().sort_values(ascending=False)
corr[0:81]

출루율                1.000000
최종출루               0.294426
가중배럴               0.284752
가중배럴카운트            0.278866
배럴속도타구수            0.275477
                     ...   
배럴각도타구수            0.111213
도실                 0.110967
6주기_타율             0.104409
평균기온대비 최고기온_max    0.103232
5주기_타율             0.100273
Name: 출루율, Length: 81, dtype: float64

In [134]:
train_id = End_tr['고유선수코드']
test_id = End_te['고유선수코드']

In [135]:
X = End_tr.copy()
X = X.loc[:,corr.index[0:81]]
X.drop(['출루율',  ],axis=1, inplace=True)
feature_names = X.columns.to_list()

X = pd.DataFrame(X)
y = End_tr.loc[:,["출루율"]]

In [136]:
train = X.copy()#.drop(['출루율'],axis=1)
test = End_te.loc[:,feature_names]
y_train = End_tr.출루율

In [137]:
features = pd.concat([train, test]).reset_index(drop=True)
features

,최종출루,가중배럴,가중배럴카운트,배럴속도타구수,7주기_출루율,평균타구속도,최종장타,배럴타구갯수,인플레이타구수,배럴타구비율,8주기_장타율,최종타율,볼넷,평균기온대비 최저기온_sum,iso,최고기온대비 최저기온_sum,희타,최저기온대비 평균기온_max,1,최저기온대비 최고기온_max,최고기온대비 평균기온_sum,평균기온(℃)_sum,고의사구,최저기온(℃)_sum,7주기_타율,최고기온(℃)_sum,8주기_출루율,8주기_타율,4주기_출루율,4주기_장타율,볼넷비율,1주기_장타율,순출루율,도루,4주기_타율,7주기_장타율,6,타수/타석,6주기_장타율,1주기_출루율,최저기온대비 평균기온_min,강수량(mm)_sum,최저기온대비 최고기온_min,최고기온(℃)_max,평균기온대비 최고기온_sum,최저기온대비 평균기온_std,강수량(mm)_skew,최저기온대비 최고기온_std,출장경기수,9,최저기온대비 평균기온_mean,5,최저기온(℃)_max,5주기_장타율,6주기_출루율,작전수행타구수,1주기_타율,평균기온(℃)_max,3,아웃플레이타구수,광주,2주기_출루율,타수,4,최저기온대비 최고기온_skew,최저기온대비 평균기온_skew,최저기온대비 최고기온_mean,8,잠실,선발,최고기온(℃)_skew,7,5주기_출루율,마산_창원,희비,배럴각도타구수,도실,6주기_타율,평균기온대비 최고기온_max,5주기_타율
0,0.336,2540,16,57,0.342105,129.312273,0.466,6,49,0.025641,0.375000,0.274,19,85.419446,0.192,63.822296,2,0.000000,12,0.000000,96.629409,2342.5,0,1567.3,0.272727,3178.1,0.310345,0.208333,0.280000,0.333333,0.073643,0.400000,0.062,9,0.250000,0.484848,16,0.906977,0.378378,0.321429,1.117073,246.4,1.243902,31.1,181.929368,0.000000,2.927740,0.000000,60,10,0.000000,15,20.5,0.555556,0.263158,2.0,0.240000,23.8,15,81,10,0.318182,234,15,0.000000,0.000000,0.000000,21,17,59,-0.652166,12,0.388889,13,0,10,5,0.243243,1.750000,0.388889
1,0.340,3940,25,66,0.181818,134.404444,0.496,11,52,0.038732,0.500000,0.282,18,89.156349,0.214,65.933062,0,0.000000,6,0.000000,103.684779,2364.9,0,1539.0,0.156250,3248.6,0.458333,0.409091,0.218750,0.275862,0.058252,0.407407,0.058,0,0.137931,0.281250,15,0.919094,0.136364,0.321429,1.119718,359.9,1.316901,32.7,202.218865,0.000000,3.230452,0.000000,63,14,0.000000,21,21.3,0.222222,0.296296,0.0,0.296296,25.5,15,92,10,0.531250,284,16,0.000000,0.000000,0.000000,12,80,73,-0.389913,18,0.216216,17,0,13,0,0.136364,1.820000,0.194444
2,0.415,2120,15,42,0.375000,132.470341,0.446,4,37,0.019802,0.483871,0.356,21,62.182890,0.090,47.675327,1,0.000000,6,0.000000,66.813649,1711.9,2,1229.9,0.285714,2246.0,0.470588,0.419355,0.000000,0.000000,0.093333,0.375000,0.059,7,0.000000,0.357143,11,0.897778,0.432432,0.500000,1.145349,302.9,1.352113,32.7,116.405702,0.000000,2.738955,0.000000,36,9,0.000000,11,21.3,0.555556,0.357143,0.0,0.375000,25.5,7,51,0,0.142857,202,10,0.000000,0.000000,0.000000,12,59,49,-0.464250,9,0.487179,6,1,9,4,0.270270,1.750000,0.472222
3,0.330,1100,7,53,0.285714,129.425620,0.350,0,46,0.000000,0.500000,0.283,13,78.901807,0.067,59.531523,8,0.000000,17,0.000000,88.821596,2003.8,0,1370.4,0.285714,2701.7,0.473684,0.375000,0.333333,0.521739,0.064356,0.500000,0.047,6,0.304348,0.314286,15,0.891089,0.333333,0.272727,1.119718,366.6,1.316901,32.7,166.775460,0.000000,2.182782,0.000000,48,11,0.000000,12,19.5,0.300000,0.250000,2.0,0.300000,25.5,16,73,15,0.358974,180,15,0.000000,0.000000,0.000000,15,56,43,-0.219519,11,0.300000,4,1,8,1,0.333333,1.820000,0.300000
4,0.294,2600,16,52,0.343750,135.678378,0.381,4,46,0.016949,0.375000,0.263,11,75.033546,0.118,57.673054,0,0.000000,10,0.000000,82.827701,1981.0,1,1415.5,0.354839,2621.6,0.309524,0.300000,0.200000,0.200000,0.043651,0.628571,0.031,11,0.200000,0.451613,15,0.936508,0.000000,0.410256,1.116505,475.1,1.257282,32.7,150.831235,0.000000,2.341841,0.000000,47,9,0.000000,18,21.3,0.000000,0.000000,3.0,0.342857,25.5,12,62,7,0.142857,236,13,0.000000,0.000000,0.000000,11,14,54,-0.718079,10,0.000000,4,4,9,3,0.000000,1.750000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,0.342,4060,28,107,0.272727,133.992522,0.399,3,89,0.009009,0.500000,0.294,21,154.840605,0.105,119.835878,0,3.551724,46,6.103448,170.536702,4050.1,1,2900.3,0.238095,5314.2,0.419355,0.400000,0.433333,0.703704,0.058333,0.394737,0.048,2,0.407407,0.404762,15,0.925000,0.480000,0.368421,1.081633,854.5,1.183673,30.4,303.203789,0.519423,4.094603,1.010848,67,14,1.580281,30,22.5,0.333333,0.466667,1.0,0

In [138]:
cat_features = features.select_dtypes(include=['object']).columns.to_list()
num_features = features.select_dtypes(exclude='object').columns.to_list()

In [139]:
if len(cat_features) > 0:
    features = pd.concat([features[num_features], pd.get_dummies(features[cat_features])], axis=1)
features

,최종출루,가중배럴,가중배럴카운트,배럴속도타구수,7주기_출루율,평균타구속도,최종장타,배럴타구갯수,인플레이타구수,배럴타구비율,8주기_장타율,최종타율,볼넷,평균기온대비 최저기온_sum,iso,최고기온대비 최저기온_sum,희타,최저기온대비 평균기온_max,1,최저기온대비 최고기온_max,최고기온대비 평균기온_sum,평균기온(℃)_sum,고의사구,최저기온(℃)_sum,7주기_타율,최고기온(℃)_sum,8주기_출루율,8주기_타율,4주기_출루율,4주기_장타율,볼넷비율,1주기_장타율,순출루율,도루,4주기_타율,7주기_장타율,6,타수/타석,6주기_장타율,1주기_출루율,최저기온대비 평균기온_min,강수량(mm)_sum,최저기온대비 최고기온_min,최고기온(℃)_max,평균기온대비 최고기온_sum,최저기온대비 평균기온_std,강수량(mm)_skew,최저기온대비 최고기온_std,출장경기수,9,최저기온대비 평균기온_mean,5,최저기온(℃)_max,5주기_장타율,6주기_출루율,작전수행타구수,1주기_타율,평균기온(℃)_max,3,아웃플레이타구수,광주,2주기_출루율,타수,4,최저기온대비 최고기온_skew,최저기온대비 평균기온_skew,최저기온대비 최고기온_mean,8,잠실,선발,최고기온(℃)_skew,7,5주기_출루율,마산_창원,희비,배럴각도타구수,도실,6주기_타율,평균기온대비 최고기온_max,5주기_타율
0,0.336,2540,16,57,0.342105,129.312273,0.466,6,49,0.025641,0.375000,0.274,19,85.419446,0.192,63.822296,2,0.000000,12,0.000000,96.629409,2342.5,0,1567.3,0.272727,3178.1,0.310345,0.208333,0.280000,0.333333,0.073643,0.400000,0.062,9,0.250000,0.484848,16,0.906977,0.378378,0.321429,1.117073,246.4,1.243902,31.1,181.929368,0.000000,2.927740,0.000000,60,10,0.000000,15,20.5,0.555556,0.263158,2.0,0.240000,23.8,15,81,10,0.318182,234,15,0.000000,0.000000,0.000000,21,17,59,-0.652166,12,0.388889,13,0,10,5,0.243243,1.750000,0.388889
1,0.340,3940,25,66,0.181818,134.404444,0.496,11,52,0.038732,0.500000,0.282,18,89.156349,0.214,65.933062,0,0.000000,6,0.000000,103.684779,2364.9,0,1539.0,0.156250,3248.6,0.458333,0.409091,0.218750,0.275862,0.058252,0.407407,0.058,0,0.137931,0.281250,15,0.919094,0.136364,0.321429,1.119718,359.9,1.316901,32.7,202.218865,0.000000,3.230452,0.000000,63,14,0.000000,21,21.3,0.222222,0.296296,0.0,0.296296,25.5,15,92,10,0.531250,284,16,0.000000,0.000000,0.000000,12,80,73,-0.389913,18,0.216216,17,0,13,0,0.136364,1.820000,0.194444
2,0.415,2120,15,42,0.375000,132.470341,0.446,4,37,0.019802,0.483871,0.356,21,62.182890,0.090,47.675327,1,0.000000,6,0.000000,66.813649,1711.9,2,1229.9,0.285714,2246.0,0.470588,0.419355,0.000000,0.000000,0.093333,0.375000,0.059,7,0.000000,0.357143,11,0.897778,0.432432,0.500000,1.145349,302.9,1.352113,32.7,116.405702,0.000000,2.738955,0.000000,36,9,0.000000,11,21.3,0.555556,0.357143,0.0,0.375000,25.5,7,51,0,0.142857,202,10,0.000000,0.000000,0.000000,12,59,49,-0.464250,9,0.487179,6,1,9,4,0.270270,1.750000,0.472222
3,0.330,1100,7,53,0.285714,129.425620,0.350,0,46,0.000000,0.500000,0.283,13,78.901807,0.067,59.531523,8,0.000000,17,0.000000,88.821596,2003.8,0,1370.4,0.285714,2701.7,0.473684,0.375000,0.333333,0.521739,0.064356,0.500000,0.047,6,0.304348,0.314286,15,0.891089,0.333333,0.272727,1.119718,366.6,1.316901,32.7,166.775460,0.000000,2.182782,0.000000,48,11,0.000000,12,19.5,0.300000,0.250000,2.0,0.300000,25.5,16,73,15,0.358974,180,15,0.000000,0.000000,0.000000,15,56,43,-0.219519,11,0.300000,4,1,8,1,0.333333,1.820000,0.300000
4,0.294,2600,16,52,0.343750,135.678378,0.381,4,46,0.016949,0.375000,0.263,11,75.033546,0.118,57.673054,0,0.000000,10,0.000000,82.827701,1981.0,1,1415.5,0.354839,2621.6,0.309524,0.300000,0.200000,0.200000,0.043651,0.628571,0.031,11,0.200000,0.451613,15,0.936508,0.000000,0.410256,1.116505,475.1,1.257282,32.7,150.831235,0.000000,2.341841,0.000000,47,9,0.000000,18,21.3,0.000000,0.000000,3.0,0.342857,25.5,12,62,7,0.142857,236,13,0.000000,0.000000,0.000000,11,14,54,-0.718079,10,0.000000,4,4,9,3,0.000000,1.750000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,0.342,4060,28,107,0.272727,133.992522,0.399,3,89,0.009009,0.500000,0.294,21,154.840605,0.105,119.835878,0,3.551724,46,6.103448,170.536702,4050.1,1,2900.3,0.238095,5314.2,0.419355,0.400000,0.433333,0.703704,0.058333,0.394737,0.048,2,0.407407,0.404762,15,0.925000,0.480000,0.368421,1.081633,854.5,1.183673,30.4,303.203789,0.519423,4.094603,1.010848,67,14,1.580281,30,22.5,0.333333,0.466667,1.0,0

In [140]:
features = pd.concat([pd.concat([train_id, test_id]).reset_index(drop=True), features], axis=1)
features

,고유선수코드,최종출루,가중배럴,가중배럴카운트,배럴속도타구수,7주기_출루율,평균타구속도,최종장타,배럴타구갯수,인플레이타구수,배럴타구비율,8주기_장타율,최종타율,볼넷,평균기온대비 최저기온_sum,iso,최고기온대비 최저기온_sum,희타,최저기온대비 평균기온_max,1,최저기온대비 최고기온_max,최고기온대비 평균기온_sum,평균기온(℃)_sum,고의사구,최저기온(℃)_sum,7주기_타율,최고기온(℃)_sum,8주기_출루율,8주기_타율,4주기_출루율,4주기_장타율,볼넷비율,1주기_장타율,순출루율,도루,4주기_타율,7주기_장타율,6,타수/타석,6주기_장타율,1주기_출루율,최저기온대비 평균기온_min,강수량(mm)_sum,최저기온대비 최고기온_min,최고기온(℃)_max,평균기온대비 최고기온_sum,최저기온대비 평균기온_std,강수량(mm)_skew,최저기온대비 최고기온_std,출장경기수,9,최저기온대비 평균기온_mean,5,최저기온(℃)_max,5주기_장타율,6주기_출루율,작전수행타구수,1주기_타율,평균기온(℃)_max,3,아웃플레이타구수,광주,2주기_출루율,타수,4,최저기온대비 최고기온_skew,최저기온대비 평균기온_skew,최저기온대비 최고기온_mean,8,잠실,선발,최고기온(℃)_skew,7,5주기_출루율,마산_창원,희비,배럴각도타구수,도실,6주기_타율,평균기온대비 최고기온_max,5주기_타율
0,2018_60558,0.336,2540,16,57,0.342105,129.312273,0.466,6,49,0.025641,0.375000,0.274,19,85.419446,0.192,63.822296,2,0.000000,12,0.000000,96.629409,2342.5,0,1567.3,0.272727,3178.1,0.310345,0.208333,0.280000,0.333333,0.073643,0.400000,0.062,9,0.250000,0.484848,16,0.906977,0.378378,0.321429,1.117073,246.4,1.243902,31.1,181.929368,0.000000,2.927740,0.000000,60,10,0.000000,15,20.5,0.555556,0.263158,2.0,0.240000,23.8,15,81,10,0.318182,234,15,0.000000,0.000000,0.000000,21,17,59,-0.652166,12,0.388889,13,0,10,5,0.243243,1.750000,0.388889
1,2018_61102,0.340,3940,25,66,0.181818,134.404444,0.496,11,52,0.038732,0.500000,0.282,18,89.156349,0.214,65.933062,0,0.000000,6,0.000000,103.684779,2364.9,0,1539.0,0.156250,3248.6,0.458333,0.409091,0.218750,0.275862,0.058252,0.407407,0.058,0,0.137931,0.281250,15,0.919094,0.136364,0.321429,1.119718,359.9,1.316901,32.7,202.218865,0.000000,3.230452,0.000000,63,14,0.000000,21,21.3,0.222222,0.296296,0.0,0.296296,25.5,15,92,10,0.531250,284,16,0.000000,0.000000,0.000000,12,80,73,-0.389913,18,0.216216,17,0,13,0,0.136364,1.820000,0.194444
2,2018_61186,0.415,2120,15,42,0.375000,132.470341,0.446,4,37,0.019802,0.483871,0.356,21,62.182890,0.090,47.675327,1,0.000000,6,0.000000,66.813649,1711.9,2,1229.9,0.285714,2246.0,0.470588,0.419355,0.000000,0.000000,0.093333,0.375000,0.059,7,0.000000,0.357143,11,0.897778,0.432432,0.500000,1.145349,302.9,1.352113,32.7,116.405702,0.000000,2.738955,0.000000,36,9,0.000000,11,21.3,0.555556,0.357143,0.0,0.375000,25.5,7,51,0,0.142857,202,10,0.000000,0.000000,0.000000,12,59,49,-0.464250,9,0.487179,6,1,9,4,0.270270,1.750000,0.472222
3,2018_61208,0.330,1100,7,53,0.285714,129.425620,0.350,0,46,0.000000,0.500000,0.283,13,78.901807,0.067,59.531523,8,0.000000,17,0.000000,88.821596,2003.8,0,1370.4,0.285714,2701.7,0.473684,0.375000,0.333333,0.521739,0.064356,0.500000,0.047,6,0.304348,0.314286,15,0.891089,0.333333,0.272727,1.119718,366.6,1.316901,32.7,166.775460,0.000000,2.182782,0.000000,48,11,0.000000,12,19.5,0.300000,0.250000,2.0,0.300000,25.5,16,73,15,0.358974,180,15,0.000000,0.000000,0.000000,15,56,43,-0.219519,11,0.300000,4,1,8,1,0.333333,1.820000,0.300000
4,2018_61353,0.294,2600,16,52,0.343750,135.678378,0.381,4,46,0.016949,0.375000,0.263,11,75.033546,0.118,57.673054,0,0.000000,10,0.000000,82.827701,1981.0,1,1415.5,0.354839,2621.6,0.309524,0.300000,0.200000,0.200000,0.043651,0.628571,0.031,11,0.200000,0.451613,15,0.936508,0.000000,0.410256,1.116505,475.1,1.257282,32.7,150.831235,0.000000,2.341841,0.000000,47,9,0.000000,18,21.3,0.000000,0.000000,3.0,0.342857,25.5,12,62,7,0.142857,236,13,0.000000,0.000000,0.000000,11,14,54,-0.718079,10,0.000000,4,4,9,3,0.000000,1.750000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,2021_79240,0.342,4060,28,107,0.272727,133.992522,0.399,3,89,0.009009,0.500000,0.294,21,154.840605,0.105,119.835878,0,3.551724,46,6.103448,170.536702,4050.1,1,2900.3,0.238095,5314.2,0.419355,0.400000,0.433333,0.703704,0.058333,0.394737,0.048,2,0.407407,0.404762,15,0.925000,0.480000,0.368421,1.081633,854.5,1.183673,30.4,303.203

In [141]:
X_train = features.query('고유선수코드 in @train_id').drop('고유선수코드', axis=1)
X_test = features.query('고유선수코드 in @test_id').drop('고유선수코드', axis=1)

In [142]:
y_train = End_tr.출루율

# 스케일링

In [143]:
num_features_tr = X_train.select_dtypes(exclude='object').columns.to_list()
num_features_te = X_test.select_dtypes(exclude='object').columns.to_list()

In [144]:
minmax = MinMaxScaler()
X_train[num_features_tr] = minmax.fit_transform(X_train[num_features_tr])
X_test[num_features_te] = minmax.transform(X_test[num_features_te])

In [145]:
X_train_sel = X_train.copy()
X_test_sel = X_test.copy()

# 데이터 분리trian_test_split

In [146]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_sel, y_train, test_size=0.2, random_state=0)

# 여러 모델에 적용

In [147]:
clfs = [
   (
        KNeighborsRegressor(),              
        {'n_neighbors': [3,5,7,9,11],        
         'weights': ['uniform','distance']}
    ),
    (
        LinearRegression(),  
        {
         }
    ),
    (
        RandomForestRegressor(random_state=0),
        {'n_estimators': [100,200,300],
         'max_depth': [3,4,5],}

    ),
    (
        GradientBoostingRegressor(random_state=0),
        {'n_estimators': [100, 200, 300],
         'learning_rate': [1, 0.1, 0.01],}

    ),
    (
        XGBRegressor(tree_method = 'hist', random_state=0),
        {'min_child_weight': range(0, 121, 20),
         'learning_rate': np.arange(0.1, 0.6, 0.1),
         'subsample': np.arange(0.5, 1.0, 0.1)}
    ),
    (
        LGBMRegressor(random_state=0),
        {'min_child_weight': range(0, 121, 20),
         'learning_rate': np.arange(0.1, 0.6, 0.1),
         'subsample': np.arange(0.5, 1.0, 0.1)}
    ),
    (
        AdaBoostRegressor(random_state=0),
        {'n_estimators': [100, 200, 300],
         'learning_rate': [1, 0.1, 0.01],}
    ),
    (
        BaggingRegressor(random_state=0),
        {'n_estimators': [100, 200, 300]}
    ),
    (
        CatBoostRegressor(random_state=0,verbose=0),
        {'learning_rate': np.arange(0.1, 0.6, 0.1),
         'depth': [3,4,5]}
    ),
    (
        SGDRegressor(random_state=0),
        {'penalty': ['l1','l2'],
         'alpha': [0.001, 0.05, 1, 100, 200],
         'max_iter': [1000, 2000, 3000]}
    ),
    (
        ExtraTreesRegressor(random_state=0),
        {'n_estimators': [100,200,300],
         'max_depth': [3,4,5,6,7]}
    ),
    (
        DecisionTreeRegressor(random_state=0),              
        {'max_depth': range(2,9,1),        
         'criterion': ['mse', 'friedman_mse', 'mae'],
         'splitter': ['best','random']}
    ),
]

clfs_tuned = []  

sscv = ShuffleSplit(n_splits = 5, random_state = 0)

for clf, param_grid in tqdm(clfs):
    start = time.time()
    rand_search = RandomizedSearchCV(clf, param_grid, n_iter=5, scoring='neg_mean_squared_error', 
                                     cv=sscv, random_state=0, n_jobs=-1)
    rand_search.fit(X_train, y_train)
    clf_name = type(clf).__name__
    clf_score = rand_search.score(X_valid, y_valid)
    print('{:30s} {:30f} {:.1f}'.format(clf_name, clf_score, time.time() - start))
    clfs_tuned.append((clf_name, rand_search, clf_score))

  8%|▊         | 1/12 [00:05<00:55,  5.00s/it]

KNeighborsRegressor                                 -0.003805 5.0
LinearRegression                                    -0.007267 0.0


 25%|██▌       | 3/12 [00:07<00:19,  2.20s/it]

RandomForestRegressor                               -0.003443 2.4


 33%|███▎      | 4/12 [00:09<00:17,  2.16s/it]

...

BaggingRegressor                                    -0.003510 3.0


 75%|███████▌  | 9/12 [00:34<00:20,  6.79s/it]

CatBoostRegressor                                   -0.004653 16.8
SGDRegressor                                        -0.004045 0.1


100%|██████████| 12/12 [00:36<00:00,  3.04s/it]

ExtraTreesRegressor                                 -0.003623 1.4
DecisionTreeRegressor                               -0.004324 0.1


In [148]:
selected = [
    'KNeighborsRegressor', 
    'LinearRegression', 
    'RandomForestRegressor', 
    'GradientBoostingRegressor',    
    'XGBRegressor',
    'LGBMRegressor',
    'AdaBoostRegressor',
    'BaggingRegressor',
    'CatBoostRegressor',
    'SGDRegressor',
    'ExtraTreesRegressor',
    'DecisionTreeRegressor',
]
models_for_ensemble = [clf for clf in clfs_tuned if clf[0] in selected]

# 스태킹

In [149]:
# Initialize 1st level estimators
selected = [
    'KNeighborsRegressor', 
    'LinearRegression', 
    'RandomForestRegressor', 
    'GradientBoostingRegressor',    
    'XGBRegressor',
    'LGBMRegressor',
    'AdaBoostRegressor',
    'BaggingRegressor',
    'CatBoostRegressor',
    'SGDRegressor',
    'ExtraTreesRegressor',
    'DecisionTreeRegressor',
]
estimators = [(name, clf) for name, clf, _ in clfs_tuned if name in selected]
              
# Initialize StackingTransformer
stack = StackingTransformer(estimators, regression=True, needs_proba=True, metric=None,
                            n_folds=3, stratified=True, shuffle=True, random_state=0)

# Fit
stack = stack.fit(X_train, y_train)

# Get your stacked features
S_train = stack.transform(X_train)
S_valid = stack.transform(X_valid)
S_test = stack.transform(X_test_sel.values)

# Use 2nd level estimator with stacked features
stacked_model = LinearRegression().fit(S_train, y_train)
print(mean_squared_error(y_valid, stacked_model.predict(S_valid)))  

tf.Tensor(0.003716897798443405, shape=(), dtype=float64)


# 예측 파일 생성

In [150]:
# Apply the stacking model to Unseen Data
stack2 = pd.DataFrame({'고유선수코드': test_id, '출루율': stacked_model.predict(S_test)})
stack2.to_csv('출루율_stacked2.csv', encoding='cp949', index=False)

stack2

,고유선수코드,출루율
257,2021_50165,0.326720
258,2021_50458,0.346923
259,2021_50506,0.399602
260,2021_50704,0.318454
261,2021_50854,0.383593
...,...,...
369,2021_79240,0.380835
370,2021_79365,0.364533
371,2021_79402,0.340412
372,2021_79456,0.359868


In [151]:
target_2021 = ['2021_76232', '2021_68050', '2021_75847', '2021_67341', '2021_79192', '2021_78224'
               , '2021_78513', '2021_76290','2021_79215', '2021_67872']

In [152]:
result = stack2.query('고유선수코드 in @target_2021').reset_index(drop = True)

In [153]:
ex = pd.read_excel(data_path_2+"/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_평가데이터.xlsx")

In [154]:
ex.iloc[1:,1:]

,PCODE,OPS,장타율,출루율
1,67341,0.912129,0.512230,0.399899
2,67872,0.749573,0.413405,0.336168
3,68050,0.933805,0.542398,0.391407
4,75847,0.827187,0.454819,0.372368
5,76232,0.929541,0.528337,0.401204
6,76290,0.837713,0.461132,0.376581
7,78224,0.882061,0.501144,0.380917
8,78513,0.840977,0.458852,0.382125
9,79192,0.888412,0.502936,0.385475
10,79215,0.861856,0.488424,0.373432


In [155]:
ex.iloc[1:11,1] = result.고유선수코드.apply(lambda x:x.split("_")[1])

In [156]:
ex.iloc[1:11,4] = result.출루율

In [157]:
result

,고유선수코드,출루율
0,2021_67341,0.399907
1,2021_67872,0.336069
2,2021_68050,0.391306
3,2021_75847,0.372195
4,2021_76232,0.401248
5,2021_76290,0.376484
6,2021_78224,0.381066
7,2021_78513,0.382178
8,2021_79192,0.385502
9,2021_79215,0.373379


In [158]:
ex

,NO.,PCODE,OPS,장타율,출루율
0,NaN,(예시),1.090000,0.680000,0.417000
1,1,67341,0.912129,0.512230,0.399907
2,2,67872,0.749573,0.413405,0.336069
3,3,68050,0.933805,0.542398,0.391306
4,4,75847,0.827187,0.454819,0.372195
5,5,76232,0.929541,0.528337,0.401248
6,6,76290,0.837713,0.461132,0.376484
7,7,78224,0.882061,0.501144,0.381066
8,8,78513,0.840977,0.458852,0.382178
9,9,79192,0.888412,0.502936,0.385502


In [162]:
ex.to_excel(data_path_2+"/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_평가데이터.xlsx", index = False)

In [163]:
pd.read_excel(data_path_2+"/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_평가데이터.xlsx")

,NO.,PCODE,OPS,장타율,출루율
0,NaN,(예시),1.090000,0.680000,0.417000
1,1,67341,0.912129,0.512230,0.399907
2,2,67872,0.749573,0.413405,0.336069
3,3,68050,0.933805,0.542398,0.391306
4,4,75847,0.827187,0.454819,0.372195
5,5,76232,0.929541,0.528337,0.401248
6,6,76290,0.837713,0.461132,0.376484
7,7,78224,0.882061,0.501144,0.381066
8,8,78513,0.840977,0.458852,0.382178
9,9,79192,0.888412,0.502936,0.385502


In [160]:
###### [76232,  68050,  75847, 67341, 79192,   78224, 78513, 76290,   79215,   67872]
######   양의지, 강백호, 최정, 이정후, 채은성, 김재환, 전준우, 김현수,  박건우, 로맥